# Анализ базы стартапа книг по подписке

**Цель проекта**

Работа делается с целью изучения базы данных купленного стартапа. Изучение особенностей данных позволит помочь сформулировать тезисы о пользе приложения для пользователей.

**Исходные данные**

База данных `PostgreSQL` 

Название: `data-analyst-final-project-db` 

Таблицы: `books` `authors` `ratings` `reviews` и `publishers`

## Импорт библиотек, подготовка соединения

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

pd.set_option('display.max_colwidth', 0)

In [2]:
db_config = {'user': 'praktikum_student', # username
'pwd': 'Sdf4$2;d-d30pp', # password
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # port
'db': 'data-analyst-final-project-db'} # db name
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# saving connector
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Первичный анализ данных

Отобразим первые пять строк всех таблиц

In [3]:
query = '''SELECT * FROM books LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''SELECT * FROM authors LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = '''SELECT * FROM ratings LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
query = '''SELECT * FROM reviews LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


In [7]:
query = '''SELECT * FROM publishers LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Структура данных соответствует описанию

## Запросы

### Книги после 2000 года

Найдем, сколько книг вышло после 1 января 2000 года

In [8]:
query = '''
SELECT COUNT(*) 
FROM books
WHERE publication_date >= '2000-01-01'
'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,count
0,821


Сколько книг в базе всего?

In [9]:
query = '''
SELECT COUNT(*) 
FROM books
'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,count
0,1000


Всего в базе 1000 книг, из них ~80 % вышло после 1 января 2000 года 

### Количество обзоров, средняя оценка на книгу

Посчитаем количество обзоров и среднюю оценку для каждой книги

In [10]:
query = '''
WITH
rating_x_book AS
(SELECT 
    book_id,
    AVG(rating) AS avg_rating
FROM ratings
GROUP BY book_id
ORDER BY avg_rating DESC),

reviews_x_book AS
(SELECT 
    book_id,
    COUNT(review_id) AS review_cnt
FROM reviews
GROUP BY book_id
ORDER BY review_cnt DESC)

SELECT
    title,
    COALESCE(review_cnt, 0) AS review_cnt,
    ROUND(avg_rating, 2) AS avg_rating_rounded
FROM books AS b
LEFT JOIN reviews_x_book ON b.book_id=reviews_x_book.book_id
LEFT JOIN rating_x_book ON b.book_id=rating_x_book.book_id
ORDER BY review_cnt DESC

'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,title,review_cnt,avg_rating_rounded
0,Twilight (Twilight #1),7,3.66
1,The Book Thief,6,4.26
2,Outlander (Outlander #1),6,4.13
3,Water for Elephants,6,3.98
4,The Da Vinci Code (Robert Langdon #2),6,3.83
...,...,...,...
995,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
996,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
997,Essential Tales and Poems,0,4.00
998,Leonardo's Notebooks,0,4.00


- Больше всего обзоров на `Twilight`, `The Book Thief`, `Outlander`, `Water for Elephants` и `The Da Vinci Code `
- Часть произведений не имеет обзоров

### Топовое издательство

Найдем издательство, выпустившее больше всех книг без учета брошюр (> 50 страниц)

In [11]:
query = '''
SELECT 
    p.publisher,
    top.books_cnt
FROM 
    publishers AS p
    RIGHT JOIN (SELECT 
                    publisher_id,
                    COUNT(book_id) AS books_cnt
                FROM books
                WHERE num_pages > 50
                GROUP BY publisher_id 
                ORDER BY books_cnt DESC
                LIMIT 1) AS top ON top.publisher_id=p.publisher_id
    
'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,publisher,books_cnt
0,Penguin Books,42


Больше всего книг выпущено `Penguin Books`: 42 книги

### Топовый автор

Найдем автора с самой высокой средней оценкой книг (возьмем только книги с количеством оценок 50+)

In [12]:
query = '''

SELECT
    author,
    author_avg_rating
FROM 
    authors AS a
    RIGHT JOIN 
    (SELECT 
        author_id,
        ROUND(AVG(avg_rating), 2) AS author_avg_rating
    FROM
        books as b 
        RIGHT JOIN
        (SELECT 
            book_id,
            COUNT(username) AS ratings_cnt,
            AVG(rating) AS avg_rating
        FROM ratings
        GROUP BY book_id
        HAVING COUNT(username) > 50
        ORDER BY avg_rating DESC) AS top ON b.book_id=top.book_id
    GROUP BY author_id
    ORDER BY author_avg_rating DESC
    LIMIT 10) AS top_x_rating ON top_x_rating.author_id=a.author_id
   
'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,author,author_avg_rating
0,J.K. Rowling/Mary GrandPré,4.28
1,J.R.R. Tolkien,4.26
2,Markus Zusak/Cao Xuân Việt Khương,4.26
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08
5,William Golding,3.90
6,J.D. Salinger,3.83
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.79
8,William Shakespeare/Paul Werstine/Barbara A. Mowat,3.79
9,Lois Lowry,3.75


Наиболее оцененные книги у `J.K. Rowling`

### Продуктивность активных пользователей

Найдем среднее количество обзоров от самых активных пользователей (количество оценок 50+)

In [13]:
query = '''

SELECT ROUND(AVG(review_cnt), 1) AS avg_review_cnt
FROM
    (SELECT
        username,
        COUNT(review_id) AS review_cnt
    FROM reviews
    WHERE username IN
        (SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating_id) > 50)
    GROUP BY username) AS top_users
   
   
'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,avg_review_cnt
0,24.3


Самые активные пользователи в среднем пишут ~24 обзора

## **Вывод**

В ходе проекта изучена база данных книжного стартапа. В базе данных хранится информация о книгах, авторах, издателях, рейтингах и обзорах книг. 
- В базе данных 1000 книг, из них вышедших после 1 января 2000 года ~ 80 % 
- Максимальное количество обзоров на книгу 6-7, есть книги без обзоров
- Наиболее обозреваемые книги -- не самые рейтинговые, их рейтинг в интервале 3.66-4.26
- Самое большое количество книг выпущено издательством `Penguin Books`, на их счету 42 книги
- Авторы с самыми высокими рейтингами книг:
    - Первое место: `J.K. Rowling`
    - Второе место делят: `J.R.R. Tolkien`, `Markus Zusak`
    - Третье место: `Louisa May Alcott`
- Самые активные пользователи написали, в среднем, по 24 обзора

Информация, полученная в ходе исследования, будет использована как основа для формирования ценностного предложения пользователям.
